# Check peak moving behaviour during optimization

In [ ]:
%load_ext autoreload
%autoreload 2
# imports
import copy
import datetime
import io
import logging
import os
import sys

import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from tqdm import tqdm

from loss_functions import final_stage_heat_loss, multi_stage_loss
from plot_utils import plot_human_lines, plot_pose
from tf_pose import common
from tf_pose.common import CocoPart
from tf_pose.estimator import PoseEstimator, TfPoseEstimator
from tf_pose.networks import get_graph_path
from utils import print_image_Stats, compare_poses, get_humans_as_lines


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)
logging.getLogger('TfPoseEstimatorRun').setLevel(logging.ERROR)
logging.getLogger('DeepExplain').setLevel(logging.ERROR)
logging.getLogger('TfPoseEstimator').setLevel(logging.ERROR)

In [ ]:
# params 432, 368
w, h = 432, 368
image_path_source = '../data/images/lea_source.jpg'
image_path_target = '../data/images/lea_target_far.jpg'
model = 'cmu'
log_dir = '../logs/'
resize_out_ratio = 2.0
image_source = common.read_imgfile(image_path_source, w, h)
image_target = common.read_imgfile(image_path_target, w, h)

summary_writer = tf.summary.FileWriter(
    log_dir + "peakMove/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
e_source = TfPoseEstimator(get_graph_path(
    model), target_size=(w, h), trt_bool=False)
e_target = TfPoseEstimator(get_graph_path(
    model), target_size=(w, h), trt_bool=False)
upsample_size = [int(e_target.target_size[1] / 8 * resize_out_ratio),
                 int(e_target.target_size[0] / 8 * resize_out_ratio)]

In [ ]:
human_source = e_source.inference(image_source, resize_to_default=(
    w > 0 and h > 0), upsample_size=resize_out_ratio)
plot_pose(image_source, human_source, e_source.heatMat)

In [ ]:
human_target = e_target.inference(image_target, resize_to_default=(
    w > 0 and h > 0), upsample_size=resize_out_ratio)
plot_pose(image_target, human_target, e_target.heatMat)

In [ ]:
source_lines = get_humans_as_lines(human_source, 400, 450)
target_lines = get_humans_as_lines(human_target, 400, 450)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)

plot_human_lines(source_lines, ax, color='r', linestyle='-', label='source')
plot_human_lines(target_lines, ax, color='g', linestyle='-', label='target')

legend_elements = [Line2D([0], [0], color='r', label='source'),
                   Line2D([0], [0], color='g', label='target'),
                   Line2D([0], [0], color='b', label='adverserial')]
ax.legend(handles=legend_elements, loc='best',  prop={'size': 20})

In [ ]:
# get all outputs as np matrix
target_np, target_np_heat, target_np_paf, target_np_peaks = e_target.persistent_sess.run(
    [e_target.tensor_output, e_target.tensor_heatMat_up,
        e_target.tensor_pafMat_up, e_target.tensor_peaks],
    feed_dict={
        e_target.tensor_image: [image_target],
        e_target.upsample_size: upsample_size
    }
)

source_np, source_np_heat, source_np_paf, source_np_peaks = e_source.persistent_sess.run(
    [e_source.tensor_output, e_source.tensor_heatMat_up,
        e_source.tensor_pafMat_up, e_source.tensor_peaks],
    feed_dict={
        e_source.tensor_image: [image_source],
        e_source.upsample_size: upsample_size
    }
)

In [ ]:
print(target_np.shape)
print(target_np_heat.shape)
print(target_np_paf.shape)
print(target_np_peaks.shape)

# for i in range(52):
#     print(np.sum(np.abs(target_np[:, :, :, i] - source_np[:, :, :, i])))


human_source = PoseEstimator.estimate_paf(
    source_np_peaks[0], source_np_heat[0], source_np_paf[0])
human_target = PoseEstimator.estimate_paf(
    target_np_peaks[0], target_np_heat[0], target_np_paf[0])


source_lines = get_humans_as_lines(human_source, 400, 450)
target_lines = get_humans_as_lines(human_target, 400, 450)


fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)

plot_human_lines(source_lines, ax, color='r', linestyle='-', label='source')
plot_human_lines(target_lines, ax, color='g', linestyle='-', label='target')

legend_elements = [Line2D([0], [0], color='r', label='source'),
                   Line2D([0], [0], color='g', label='target')]
ax.legend(handles=legend_elements, loc='best',  prop={'size': 20})

In [ ]:
def gen_heat_plot(heatmap):

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(1, 1, 1)

    ax.imshow(heatmap, cmap='hot')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    return buf


EPS = 5000
STEPS = 100
STAGE_INDEX = 6  # range from 2 to 6
ONLY_HEAT_MAP_AS_LOSS = False


if not "adv_image" in vars():
    adv_image = copy.deepcopy(image_source)
target_heat, grad_func, loss_func = [None, None, None]

human_source = PoseEstimator.estimate_paf(
    source_np_peaks[0], source_np_heat[0], source_np_paf[0])
human_target = PoseEstimator.estimate_paf(
    target_np_peaks[0], target_np_heat[0], target_np_paf[0])


if ONLY_HEAT_MAP_AS_LOSS:
    target_heat = tf.compat.v1.placeholder(
        tf.float32, shape=(None, None, None, 19))
    grad_func, loss_func = final_stage_heat_loss(e_source, target_heat)
else:

    target_heat = tf.compat.v1.placeholder(
        tf.float32, shape=(None, None, None, 19))
    target_paf = tf.compat.v1.placeholder(
        tf.float32, shape=(None, None, None, 38))
    grad_func, loss_func = multi_stage_loss(
        e_source, target_heat, target_paf, STAGE_INDEX)

for i in tqdm(range(STEPS)):
    # compute gradient
    gradient, gn_summ = e_source.persistent_sess.run(
        [grad_func, loss_func],
        feed_dict={
            target_heat: target_np[:, :, :, CocoPart.RWrist.value] if ONLY_HEAT_MAP_AS_LOSS else target_np[:, :, :, :19],
            target_paf: target_np[:, :, :, 19:],
            e_source.tensor_image: [adv_image],
            e_source.upsample_size: upsample_size
        }
    )
    summary_writer.add_summary(gn_summ, i)
    if i % 2 == 0:
        human_adv = e_source.inference(adv_image, resize_to_default=(
            w > 0 and h > 0), upsample_size=resize_out_ratio)
        # Prepare the plot
        plot_buf = gen_heat_plot(e_source.heatMat[:, :, CocoPart.RWrist.value])

        # Convert PNG buffer to TF image
        tf_image = tf.image.decode_png(plot_buf.getvalue(), channels=4)

        # Add the batch dimension
        tf_image = tf.expand_dims(tf_image, 0)

        # Add image summary
        summary_op = tf.summary.image(f"heatmap_wrist_step{i}", tf_image)
        summary_img = e_source.persistent_sess.run(summary_op)
        # Write summary

        summary_writer.add_summary(summary_img)
        summary_writer.flush()

    ### ================= UPDATE STEP ================= ###
    #adv_image = adv_image - EPS * np.sign(gradient[0])
    scaled_gradient = (EPS * gradient[0])
    adv_image = adv_image - scaled_gradient
    adv_image = np.clip(adv_image, 0, 255)

summary_writer.flush()

human_adv = e_source.inference(adv_image, resize_to_default=(
    w > 0 and h > 0), upsample_size=resize_out_ratio)

print_image_Stats(image_source)
print_image_Stats(adv_image)
total_diff, most_moved_part = compare_poses(human_source[0], human_adv[0])
print(
    f'The total diff is {total_diff} and the part moved the most is the {most_moved_part}')